In [1]:
import os
import sys
import time
import random
import warnings
import collections
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.experimental import enable_hist_gradient_boosting, enable_halving_search_cv  
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, HalvingRandomSearchCV 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor, HistGradientBoostingRegressor 


# Import required modules
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# from sklearn.linear_model import Ridge, Lasso, BayesianRidge, ElasticNet
from sklearn.preprocessing import OneHotEncoder

sys.path.append('../../src')
import cb_utils
import cb_model_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

/Users/bp/miniforge3/envs/data-analytics/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


### Load raw data

In [2]:
query = f"select * from junk.ml_training_samples_20250210;"
df = cb_utils.sql_query_to_df(query, use_cache=True, source='msh_analytics')

Pulling query from db


In [3]:
df.head()

,payer_id,period_number,member_id,pre_elg_days,age_ft,is_male_ft,is_female_ft,ip_tc_pre_pmpm_ft,ed_tc_pre_pmpm_ft,snf_tc_pre_pmpm_ft,icf_tc_pre_pmpm_ft,hh_tc_pre_pmpm_ft,out_tc_pre_pmpm_ft,pro_tc_pre_pmpm_ft,hcbs_tc_pre_pmpm_ft,sphs_tc_pre_pmpm_ft,amb_tc_pre_pmpm_ft,dme_tc_pre_pmpm_ft,hosp_tc_pre_pmpm_ft,dialysis_ddos_pre_pmpm_ft,pulmonar_ddos_pre_pmpm_ft,copd_ddos_pre_pmpm_ft,chf_ddos_pre_pmpm_ft,heart_ddos_pre_pmpm_ft,cancer_ddos_pre_pmpm_ft,ckd_ddos_pre_pmpm_ft,esrd_ddos_pre_pmpm_ft,hyperlipid_ddos_pre_pmpm_ft,diab_ddos_pre_pmpm_ft,alzh_ddos_pre_pmpm_ft,dementia_ddos_pre_pmpm_ft,neurocognitive_ddos_pre_pmpm_ft,stroke_ddos_pre_pmpm_ft,hypertension_ddos_pre_pmpm_ft,fall_ddos_pre_pmpm_ft,transplant_ddos_pre_pmpm_ft,liver_ddos_pre_pmpm_ft,hippfract_ddos_pre_pmpm_ft,depression_ddos_pre_pmpm_ft,psychosis_ddos_pre_pmpm_ft,drug_ddos_pre_pmpm_ft,alcohol_ddos_pre_pmpm_ft,paralysis_ddos_pre_pmpm_ft,hemophilia_ddos_pre_pmpm_ft,pressure_ulcer_ddos_pre_pmpm_ft,tbi_ddos_pre_pmpm_ft,obese_ddos_pre_pmpm_ft,post_elig_days,tc_tg,tc_pmpm_tg
0,81,19,20362,181,68.0,1,0,0.00,0.0,0.0,0.0,0.00,6.30,119.95,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.17,0.00,0.0,0.0,0.0,0.00,0.17,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365,676.96,5.56
1,81,19,20363,181,90.0,0,1,2530.75,0.0,3756.4,0.0,387.25,0.00,434.10,0.0,0.0,100.83,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.5,0.0,0.0,0.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365,27889.78,229.23
2,81,19,20368,181,76.0,0,1,0.00,0.0,0.0,0.0,1240.69,0.00,69.43,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.17,0.17,0.17,0.17,0.0,0.0,0.0,0.00,0.17,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365,13185.44,108.37
3,81,19,20371,181,73.0,1,0,0.00,0.0,0.0,0.0,0.00,1.21,123.45,0.0,0.0,0.00,7.05,0.0,0.0,0.0,0.0,0.33,1.49,0.0,0.00,0.00,0.99,0.00,0.0,0.0,0.0,0.17,0.33,0.0,0.0,0.0,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365,31207.91,256.50
4,81,19,20374,181,71.0,1,0,0.00,0.0,0.0,0.0,583.64,0.00,46.70,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.17,0.00,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,365,4454.73,36.61


In [4]:
df.shape

(1765592, 50)

In [15]:
df.member_id.nunique()



Removing 2058 samples with negative tc_pmpm_tg values
Updated dataframe shape: (1763534, 52)


In [16]:
# add pmpm target bucket
# Define the bucket boundaries
bucket_boundaries = [0, 55, 74, 106, 155, 235, 291, 495, 886, float('inf')]

# Define the bucket labels
bucket_labels = ['< 55', '55-74', '74-106', '106-155', '155-235', '235-291', '291-495', '495-886', '> 886']

# Create the bucket column
df['pmpm_bucket'] = pd.cut(df['tc_pmpm_tg'], bins=bucket_boundaries, labels=bucket_labels, right=False)
# Create a numeric bucket ID column based on the bucket index
# Map each bucket label to its index (0-8)
bucket_to_id_map = {label: idx for idx, label in enumerate(bucket_labels)}
df['pmpm_bucket_id'] = df['pmpm_bucket'].map(bucket_to_id_map)

# Verify the mapping
print("\nBucket ID mapping:")
for label, idx in bucket_to_id_map.items():
    print(f"{label}: {idx}")

# Display the distribution of buckets
bucket_counts = df['pmpm_bucket'].value_counts().sort_index()
print("PMPM Bucket Distribution:")
print(bucket_counts)
print("\nPercentage Distribution:")
print(bucket_counts / len(df) * 100)




Bucket ID mapping:
< 55: 0
55-74: 1
74-106: 2
106-155: 3
155-235: 4
235-291: 5
291-495: 6
495-886: 7
> 886: 8
PMPM Bucket Distribution:
< 55       1124697
55-74       107461
74-106      110462
106-155     107480
155-235      99434
235-291      43578
291-495      89993
495-886      54149
> 886        26280
Name: pmpm_bucket, dtype: int64

Percentage Distribution:
< 55       63.775181
55-74       6.093503
74-106      6.263673
106-155     6.094581
155-235     5.638338
235-291     2.471061
291-495     5.102992
495-886     3.070482
> 886       1.490190
Name: pmpm_bucket, dtype: float64


### Train test split

In [17]:
train_pct = 0.8
# Get unique member IDs and randomly split them
all_members = df.member_id.unique()
np.random.seed(42)
train_members = np.random.choice(all_members, size=int(len(all_members) * train_pct), replace=False)
test_members = np.setdiff1d(all_members, train_members)

# Split dataframes based on member lists
train_df = df[df.member_id.isin(train_members)].copy()
test_df = df[df.member_id.isin(test_members)].copy()

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nUnique members in train: {train_df.member_id.nunique()}")
print(f"Unique members in test: {test_df.member_id.nunique()}")



Train shape: (1410049, 52)
Test shape: (353485, 52)

Unique members in train: 116820
Unique members in test: 29205


In [18]:
# Get feature columns ending in _ft
feature_cols = [col for col in train_df.columns if col.endswith('_ft')]
target_col = 'pmpm_bucket_id'

# Create feature matrix X and target vector y
X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]


In [21]:
# Initialize HistGradientBoostingRegressor
# model = HistGradientBoostingRegressor(random_state=42)
# Since we're predicting bucket IDs which are categorical,
# we should use HistGradientBoostingClassifier instead of Regressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
import numpy as np

# Convert target to categorical if it's not already
y_train = y_train.astype('int')
y_test = y_test.astype('int')

# Replace the regressor with a classifier
model = HistGradientBoostingClassifier(
    random_state=42,
    # Explicitly tell the model that our target is categorical
    # This enables the model to use specialized handling for classification
    # loss='categorical_crossentropy'
)

# Initialize cross-validation scores list
cv_scores = []

# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    # Split data
    X_fold_train = X_train.iloc[train_idx]
    X_fold_val = X_train.iloc[val_idx]
    y_fold_train = y_train.iloc[train_idx]
    y_fold_val = y_train.iloc[val_idx]
    
    # Train model
    model.fit(X_fold_train, y_fold_train)
    
    # Make predictions
    y_pred = model.predict(X_fold_val)
    
    # Calculate R2 score
    acc = accuracy_score(y_fold_val, y_pred)
    cv_scores.append(acc)
    
    print(f"Fold {fold + 1} Accuracy: {acc:.4f}")

print(f"\nMean CV Accuracy: {np.mean(cv_scores):.4f}")
print(f"Std CV Accuracy: {np.std(cv_scores):.4f}")

Fold 1 Accuracy: 0.6569
Fold 2 Accuracy: 0.6566
Fold 3 Accuracy: 0.6565
Fold 4 Accuracy: 0.6564
Fold 5 Accuracy: 0.6562

Mean CV Accuracy: 0.6565
Std CV Accuracy: 0.0002


In [22]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_iter': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [20, 50, 100]
}
# grid_search = GridSearchCV(HistGradientBoostingRegressor(random_state=42),
grid_search = RandomizedSearchCV(HistGradientBoostingClassifier(random_state=42),
                          param_distributions=param_grid,
                          cv=5,
                          scoring='r2')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

{'min_samples_leaf': 100, 'max_iter': 200, 'max_depth': 5, 'learning_rate': 0.1}


In [23]:
# try model with best params
# Initialize HistGradientBoostingRegressor
model = HistGradientBoostingClassifier(random_state=42, **grid_search.best_params_)

# Initialize cross-validation scores list
accuracy_scores = []
balanced_accuracy_scores = []

# Perform k-fold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    # Split data
    X_fold_train = X_train.iloc[train_idx]
    X_fold_val = X_train.iloc[val_idx]
    y_fold_train = y_train.iloc[train_idx]
    y_fold_val = y_train.iloc[val_idx]
    
    
    # Train model
    model.fit(X_fold_train, y_fold_train)
    
    # Make predictions
    y_pred = model.predict(X_fold_val)
    
    # Calculate accuracy and balanced accuracy scores
    acc = accuracy_score(y_fold_val, y_pred)
    bal_acc = balanced_accuracy_score(y_fold_val, y_pred)
    accuracy_scores.append(acc)
    balanced_accuracy_scores.append(bal_acc)
    
    print(f"Fold {fold + 1} Accuracy: {acc:.4f}, Balanced Accuracy: {bal_acc:.4f}")

print(f"\nMean CV Accuracy: {np.mean(accuracy_scores):.4f}")
print(f"Std CV Accuracy: {np.std(accuracy_scores):.4f}")
print(f"Mean CV Balanced Accuracy: {np.mean(balanced_accuracy_scores):.4f}")
print(f"Std CV Balanced Accuracy: {np.std(balanced_accuracy_scores):.4f}")


Fold 1 Accuracy: 0.6563, Balanced Accuracy: 0.1843
Fold 2 Accuracy: 0.6556, Balanced Accuracy: 0.1837
Fold 3 Accuracy: 0.6559, Balanced Accuracy: 0.1846
Fold 4 Accuracy: 0.6558, Balanced Accuracy: 0.1831
Fold 5 Accuracy: 0.6548, Balanced Accuracy: 0.1823

Mean CV Accuracy: 0.6557
Std CV Accuracy: 0.0005
Mean CV Balanced Accuracy: 0.1836
Std CV Balanced Accuracy: 0.0008


In [24]:
# Train the model on the entire training set
model = HistGradientBoostingClassifier(random_state=42, **grid_search.best_params_)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_test = model.predict(X_test)

# Evaluate the model on the test set
acc_test = accuracy_score(y_test, y_pred_test)
bal_acc_test = balanced_accuracy_score(y_test, y_pred_test)
print(f"Test Set Accuracy: {acc_test:.4f}, Balanced Accuracy: {bal_acc_test:.4f}")

Test Set Accuracy: 0.6438, Balanced Accuracy: 0.1504


In [14]:

import pickle

# --- Ensure this cell runs after your model is trained ---

# !!! IMPORTANT: Replace 'model' with the variable name of your trained scikit-learn model !!!
model_to_save = model
model_filename = 'trained_model_20250328.pkl'

print(f"Saving model to {model_filename}...")
# Use protocol=5 for potentially better performance with large numpy arrays (requires Python 3.8+)
with open(model_filename, 'wb') as file:
    pickle.dump(model_to_save, file, protocol=5)
print("Model saved successfully.")

Saving model to trained_model_20250328.pkl...
Model saved successfully.


In [15]:
## Load the model 
import pickle

# --- This cell loads the model saved previously ---

model_filename = 'trained_model_20250328.pkl'
loaded_model = None # Initialize variable

print(f"Loading model from {model_filename}...")
with open(model_filename, 'rb') as file:
    loaded_model = pickle.load(file)
print("Model loaded successfully.")


# You can also verify the model type (optional)
print(f"Loaded model type: {type(loaded_model)}")

Loading model from trained_model_20250328.pkl...
Model loaded successfully.
Loaded model type: <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingRegressor'>


In [16]:

query = f"select * from junk.ml_training_samples_20250326;"
new_df = cb_utils.sql_query_to_df(query, use_cache=True, source='msh_analytics')

Pulling query from db


In [17]:
new_df.shape

(19259, 49)

In [18]:
# Make predictions on the test set
y_pred_test = model.predict(new_df[feature_cols])

# Evaluate the model on the test set
r2_test = r2_score(new_df[target_col], y_pred_test)
print(f"Test Set R² Score: {r2_test:.4f}")



Test Set R² Score: 0.1277


In [21]:
# Calculate various metrics
mse = mean_squared_error(new_df[target_col], y_pred_test)
rmse = np.sqrt(mse)
mae = mean_absolute_error(new_df[target_col], y_pred_test)
mape = mean_absolute_percentage_error(new_df[target_col], y_pred_test)

print(f"RMSE: ${rmse:.2f}")
print(f"MAE: ${mae:.2f}")
print(f"MAPE: {mape:.2%}")

RMSE: $305.76
MAE: $119.72
MAPE: 771037041091850240.00%


In [22]:
output_df = new_df.assign(
    raw_pred=y_pred_test,
    pred_percentile=lambda df: df['raw_pred'].rank(pct=True) * 100
)

In [ ]:
output_df.head()
output_columns = ['payer_id', 'member_id', 'tc_pmpm_tg', 'raw_pred', 'pred_percentile']
output_df[output_columns].to_csv('../data/cost_preds_output_20250328.csv', index=False)